# clone Dataset from github

# install pycm pkg for ConfusionMatrix

In [1]:
! git clone https://github.com/akramtahmasebi/EFS-DNN_data.git
! pip install pycm 

Cloning into 'EFS-DNN_data'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 12 (delta 2), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (12/12), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 66 kB 4.1 MB/s 
     |████████████████████████████████| 590 kB 41.4 MB/s 


# import library & and python pkg

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sea
import lightgbm as lgb
from pycm import ConfusionMatrix
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold
import time
from pathlib import Path
import logging


import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

#  Take a look at the data 


In [3]:
pd.read_csv('EFS-DNN_data/kdd99_raw.csv').head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [4]:
pd.read_csv('EFS-DNN_data/attack_type.csv')

,attack,type
0,normal,normal
1,back,dos
2,buffer_overflow,u2r
3,ftp_write,r2l
4,guess_passwd,r2l
5,imap,r2l
6,ipsweep,probe
7,land,dos
8,loadmodule,u2r
9,multihop,r2l


In [5]:
pd.read_csv('EFS-DNN_data/nslkdd_raw.csv').head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [6]:
pd.read_csv('EFS-DNN_data/UNSW_NB15_raw.csv').head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,...,1,1,1,0,0,0,1,1,0,normal
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,...,1,1,2,0,0,0,1,6,0,normal
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,...,1,1,3,0,0,0,2,6,0,normal
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,1,3,1,1,0,2,1,0,normal
4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,...,2,1,40,0,0,0,2,39,0,normal


# preprocess Data 

In [7]:
manula_args = 2 # if you want categorize between 0 and 1  use int 2 and for categorize with attac_type use int 5

df = pd.read_csv('EFS-DNN_data/kdd99_raw.csv')

features = df.iloc[:, :-1]
target = df.iloc[:, -1]

numerical_features = [x for x in features.columns if features[x].dtype == np.int64 or features[x].dtype == np.float64]
categorical_features = [x for x in features.columns if features[x].dtype == object]

lbe = LabelEncoder()
for feat in categorical_features:
    df[feat] = lbe.fit_transform(df[feat])

mms = MinMaxScaler()
df[numerical_features] = mms.fit_transform(df[numerical_features])

if manula_args == 2:
    mask = target == 'normal'
    target[mask] = 0
    target[~mask] = 1
elif manula_args == 5:
    attack_type = pd.read_csv('EFS-DNN_data/attack_type.csv')
    attack2type = dict()
    for attack, type in attack_type.values:
        attack2type[attack] = type
    type2idx = {'normal': 0, 'dos': 1, 'probe': 2, 'r2l': 3, 'u2r': 4}
    df.iloc[:, -1] = target.apply(lambda x: type2idx[attack2type[x]])

df.to_csv('_pre.csv', index=False)

# define loger def

In [8]:
def set_logger(dataset, classes, threshold, n_lgb, r_sample):
    task_time = time.strftime("%Y-%m-%d %H:%M", time.localtime())
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    Path("log/").mkdir(parents=True, exist_ok=True)
    fh = logging.FileHandler(f'log/{dataset}_{classes}_{threshold}_{n_lgb}_{r_sample}_{task_time}.log')
    fh.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.WARN)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    logger.addHandler(fh)
    logger.addHandler(ch)

    return logger

# define NeuralNet and linear with torch pkg

In [9]:
def linear(inp, out, dropout):
    """
    linear model module by nn.sequential
    :param inp: int, linear model input dimensio
    :param out: int, linear model output dimension
    :param dropout: float dropout probability for linear layer
    :return: tensor
    """
    return nn.Sequential(
        nn.BatchNorm1d(inp),
        nn.Linear(inp, out),
        nn.LeakyReLU(),
        nn.Dropout(dropout)
    )


class NeuralNet(nn.Module):
    def __init__(self, deep_columns_idx, embedding_columns_dict, hidden_layers, dropouts, output_dim):
        """
        init parameters
        :param deep_columns_idx: dict include column name and it's index
            e.g. {'age': 0, 'career': 1,...}
        :param embedding_columns_dict: dict include categories columns name and number of unique val and embedding dimension
            e.g. {'age':(10, 32),...}
        :param hidden_layers: number of hidden layers
        :param deep_columns_idx: dict of columns name and columns index
        :param dropouts: list of float each hidden layers dropout len(dropouts) == hidden_layers - 1
        """
        super(NeuralNet, self).__init__()
        self.embedding_columns_dict = embedding_columns_dict
        self.deep_columns_idx = deep_columns_idx
        for key, val in embedding_columns_dict.items():
            setattr(self, 'dense_col_' + key, nn.Embedding(val[0], val[1]))
        embedding_layer = 0
        for col in self.deep_columns_idx.keys():
            if col in embedding_columns_dict:
                embedding_layer += embedding_columns_dict[col][1]
            else:
                embedding_layer += 1
        self.layers = nn.Sequential()
        hidden_layers = [embedding_layer] + hidden_layers
        dropouts = [0.0] + dropouts
        for i in range(1, len(hidden_layers)):
            self.layers.add_module(
                'hidden_layer_{}'.format(i - 1),
                linear(hidden_layers[i - 1], hidden_layers[i], dropouts[i - 1])
            )
        self.layers.add_module('last_linear', nn.Linear(hidden_layers[-1], output_dim))

    def forward(self, x):
        emb = []
        continuous_cols = [col for col in self.deep_columns_idx.keys() if col not in self.embedding_columns_dict]
        for col, _ in self.embedding_columns_dict.items():
            if col not in self.deep_columns_idx:
                raise ValueError("ERROR column name may be your deep_columns_idx dict is not math the"
                                 "embedding_columns_dict")
            else:
                idx = self.deep_columns_idx[col]
                emb.append(getattr(self, 'dense_col_' + col)(x[:, idx].long()))

        for col in continuous_cols:
            idx = self.deep_columns_idx[col]
            emb.append(x[:, idx].view(-1, 1).float())
        embedding_layers = torch.cat(emb, dim=1)
        out = self.layers(embedding_layers)
        return out


# define get_feature_importance def with lightGBM 

In [10]:
def get_feature_importance(features, target, num_lgb, r_sample):
    feature_importance = np.zeros([num_lgb, features.shape[1]])
    for i in range(num_lgb):
        clf = lgb.sklearn.LGBMClassifier(subsample=r_sample, subsample_freq=1, random_state=i)
        clf.fit(features, target)
        feature_importance[i] = clf.feature_importances_

    return feature_importance

# train_supervised 

define Variables

In [11]:
DATA = 'kdd99'              # Dataset name (eg. kdd99 or nslkdd)
GPU = 0                     # Idx for the gpu to use
SEED = 0                    # Random Seed
N_LGB = 3                   # The number of LightGBM
LR = 1e-4                   # Learning rate
CLASSES = 2                 # The number of classes
DROPOUT = 0.1               # Dropout probability
R_SAMPLE = 0.6              # The ratio of sampled instances
EPOCH = 30                  # Number of epochs 
BS = 1024                   # The batch size
THRESHOLD = 0.9             # The threshold in feature selection phase

define class and def for trainig

In [12]:
class IDSDataset(Dataset):
    def __init__(self, features, target, transform=None):
        self.features = features
        self.target = target

    def __len__(self):
        return len(self.target)

    def __getitem__(self, item):
        return self.features[item], self.target[item]

In [13]:
def print_results(logger, acc, f1, fpr, tpr, auc, inf_time):
    classes = len(list(acc.keys())) - 1
    if classes == 2:
        logger.info('Accuracy: {:.4f}, FPR: {:.4f}, TPR: {:.4f}, AUC: {:.4f}, F1: {:.4f}'.format(
            acc[2], fpr[2], tpr[2], auc[2], f1[2]
        ))
    elif classes == 5:
        idx2type = {0: 'Normal', 1: 'DOS', 2: 'Probe', 3: 'R2L', 4: 'U2R', 5: 'Overall'}
        for idx, type in idx2type.items():
            logger.info('Type: {}, Accuracy: {:.4f}, FPR: {:.4f}, TPR: {:.4f}, AUC: {:.4f}, F1: {:.4f}'.format(
                type, acc[idx], fpr[idx], tpr[idx], auc[idx], f1[idx]
            ))
    logger.info('Inference time: {:.8f}'.format(inf_time))

def train(dataloader, model, loss_fn, optim, device):
    model.train()
    m_loss = []

    for X, y in dataloader:
        optim.zero_grad()

        X, y = X.to(device), y.to(device)
        pred_y = model(X)
        loss = loss_fn(pred_y, y)

        loss.backward()
        optim.step()
        m_loss.append(loss.item())

    return np.mean(m_loss)

def eval(dataloader, model, classes, device):
    model.eval()
    m_acc, m_tpr, m_fpr, m_auc, m_f1 = [], [], [], [], []
    d_acc, d_tpr, d_fpr, d_auc, d_f1 = {}, {}, {}, {}, {}
    m_inf_time = []
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)

            eval_start = time.time()
            y_pred = torch.softmax(model(X).detach().cpu(), dim=1).numpy()
            m_inf_time.append(time.time() - eval_start)

            y_l = y_pred.argmax(1)
            cm = ConfusionMatrix(actual_vector=y.numpy(), predict_vector=y_l)
            m_acc.append(cm.ACC)
            m_tpr.append(cm.TPR)
            m_fpr.append(cm.FPR)
            m_f1.append(cm.F1)
            m_auc.append(cm.AUC)

    for k in range(classes):
        d_acc[k] = np.mean([d[k] if d[k] != 'None' else 0 for d in m_acc if k in set(d.keys())])
        d_f1[k] = np.mean([d[k] if d[k] != 'None' else 0 for d in m_f1 if k in set(d.keys())])
        d_fpr[k] = np.mean([d[k] if d[k] != 'None' else 0 for d in m_fpr if k in set(d.keys())])
        d_tpr[k] = np.mean([d[k] if d[k] != 'None' else 0 for d in m_tpr if k in set(d.keys())])
        d_auc[k] = np.mean([d[k] if d[k] != 'None' else 0 for d in m_auc if k in set(d.keys())])
    d_acc[classes] = np.mean(list(d_acc.values()))
    d_f1[classes] = np.mean(list(d_f1.values()))
    d_fpr[classes] = np.mean(list(d_fpr.values()))
    d_tpr[classes] = np.mean(list(d_tpr.values()))
    d_auc[classes] = np.mean(list(d_auc.values()))

    return d_acc, d_f1, d_fpr, d_tpr, d_auc, np.sum(m_inf_time)

In [14]:
def run(logger):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    device_string = 'cuda' if torch.cuda.is_available() else 'cpu' # 'cuda:{}'.format(GPU) if torch.cuda.is_available() and GPU >= 0 else 'cpu'
    print(f'Model is trained in {device_string}')
    device = torch.device(device_string)

    df = pd.read_csv('_pre.csv')

    feat_select_start = time.time()
    feats = df.iloc[:, :-1]
    target = df.iloc[:, -1]
    #print('categorical features are ', cate_feats)
    #print('numerical features are ', num_feats)

    feat_imp = get_feature_importance(feats.values, target.values, N_LGB, R_SAMPLE)
    feat_imp = feat_imp.sum(0)
    rank = np.argsort(-feat_imp)
    feat_imp = -(np.sort(-feat_imp) / feat_imp.sum())
    feat_imp = feat_imp.cumsum()
    num_in_feat = (np.where(feat_imp > THRESHOLD)[0][0]) + 1

    feats = feats.iloc[:, rank[:num_in_feat]]
    feat_select_time = time.time() - feat_select_start

    cate_feats = [x for x in feats.columns if feats[x].dtype == np.int64]
    num_feats = [x for x in feats.columns if feats[x].dtype == np.float64]
    feat_dict = {feat: idx for idx, feat in enumerate(feats.columns)}

    embedding_dim = 5
    embedding_feat = {feat: (feats[feat].value_counts().count(), embedding_dim) for feat in cate_feats}

    efsdnn = NeuralNet(feat_dict, embedding_feat, [512, 512, 512], [DROPOUT, DROPOUT, DROPOUT], CLASSES).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(efsdnn.parameters(), lr=LR)

    for i in range(EPOCH):
        if DATA == 'UNSW_NB15':
            x_train, x_test, y_train, y_test = train_test_split(feats.values, target.values, train_size=0.68, shuffle=False)
        else:
            x_train, x_test, y_train, y_test = train_test_split(feats.values, target.values, train_size=0.8, shuffle=True)
        # x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.875, shuffle=True)
        train_dataset = IDSDataset(x_train, y_train)
        # val_dataset = IDSDataset(x_val, y_val)
        test_dataset = IDSDataset(x_test, y_test)
        train_dataloader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
        # val_dataloader = DataLoader(val_dataset, batch_size=BS, shuffle=True)
        test_dataloader = DataLoader(test_dataset, batch_size=BS, shuffle=True)

        train_start = time.time()
        loss = train(train_dataloader, efsdnn, loss_fn, optim, device)
        train_time = time.time() - train_start + feat_select_time

        logger.info(f'------------Epoch: {i + 1}/{EPOCH}------------')
        logger.info(f'The number of selected features is {num_in_feat}')
        logger.info('Training time: {:.8f}'.format(train_time))

        # logger.info('---------------Validation-----------------')
        # val_acc, val_f1, val_fpr, val_tpr, val_auc, val_inf_time = eval(val_dataloader, efsdnn, CLASSES, device)
        # print_results(logger, val_acc, val_f1, val_fpr, val_tpr, val_auc, val_inf_time)

        logger.info('---------------Test-----------------')
        test_acc, test_f1, test_fpr, test_tpr, test_auc, test_inf_time = eval(test_dataloader, efsdnn, CLASSES, device)
        print_results(logger, test_acc, test_f1, test_fpr, test_tpr, test_auc, test_inf_time)

In [15]:
logger = set_logger(DATA, CLASSES, THRESHOLD, N_LGB, R_SAMPLE) 

run(logger)

Model is trained in cuda


INFO:root:------------Epoch: 1/30------------
INFO:root:The number of selected features is 19
INFO:root:Training time: 15.87719941
INFO:root:---------------Test-----------------
INFO:root:Accuracy: 0.9972, FPR: 0.0031, TPR: 0.9969, AUC: 0.9969, F1: 0.9971
INFO:root:Inference time: 0.03950810
INFO:root:------------Epoch: 2/30------------
INFO:root:The number of selected features is 19
INFO:root:Training time: 15.69242525
INFO:root:---------------Test-----------------
INFO:root:Accuracy: 0.9970, FPR: 0.0033, TPR: 0.9967, AUC: 0.9967, F1: 0.9969
INFO:root:Inference time: 0.03885150
INFO:root:------------Epoch: 3/30------------
INFO:root:The number of selected features is 19
INFO:root:Training time: 15.68352699
INFO:root:---------------Test-----------------
INFO:root:Accuracy: 0.9984, FPR: 0.0017, TPR: 0.9983, AUC: 0.9983, F1: 0.9984
INFO:root:Inference time: 0.03976870
INFO:root:------------Epoch: 4/30------------
INFO:root:The number of selected features is 19
INFO:root:Training time: 15